In [3]:

import pandas as pd
import os
import pandas as pd
from datetime import datetime

# This function finds files in a given folder that contain a specific substring in their names.
def find_relevant_files(folder_path, text1):
    relevant_files = []
    for file_name in os.listdir(folder_path):
        if text1 in file_name:
            relevant_files.append(file_name)
    return relevant_files
# *******************  END Function  *************************

# This function gets test information from file name
# In this case it extracts the serial number, target, and height
def parse_file_name(file_name):
    SN = file_name[0:5]
    # target = "Brush" if "BRUSH" in file_name.upper() else "F28P"
    target = "Brush"
    if '1p' in file_name:
        Ht = 2.50
    elif '2p' in file_name:
        Ht = 2.56
    elif '3p' in file_name:
        Ht = 2.62
    else:
        Ht = 2.43  # default height
    #if "brush" in file_name:
    #    target= "Brush"
    #elif "F28P" in file_name:
    #    target= "F28P"
    return SN, target, Ht
# *******************  END Function  *************************



# This function gets the creation date of a file and formats it as 'YYYY-MM-DD HH:MM:SS'
def get_file_creation_date(file_path):
    creation_time = os.path.getctime(file_path)
    creation_date = datetime.fromtimestamp(creation_time).strftime('%Y-%m-%d %H:%M:%S')
    return creation_date
# *******************  END Function  *************************




In [4]:
# This function reads the Peak power from cather testing: 
# In this case, 14th line in the 2nd column
def get_peak_pow(fpath):
    
    # Open the file and read line 14 (index 13)
    with open(fpath, 'r') as file:
        lines = file.readlines()
        line_14 = lines[13]  # Line 14 is at index 13

    # Split the line by comma to get columns
    columns = line_14.strip().split(',')

    # Get the 2nd column (index 1)
    aa = columns[1]

    print("Value in 2nd column of line 14:", aa)
    return aa
# *******************  END Function  *************************

# This function combines previous functions, gets data from all files and saves it in a summary csv file
def read_and_store_data(folder_path, text1):
    all_data = []
    relevant_files = find_relevant_files(folder_path, text1)
    
    for file_name in relevant_files:
        print(file_name)
        file_path = os.path.join(folder_path, file_name)

        # Read Peak Power
        peak_pow= get_peak_pow(file_path)

        # Get SN, target, and date of file creation from file name
        SN, target, Ht = parse_file_name(file_name)
        creation_date = get_file_creation_date(file_path)
        
        # Add all these to a dataframe
        df = pd.DataFrame({
            'SN': [SN],
            'target': [target],
            'DoF': [creation_date],
            'Ht': [Ht],
            'peak_pow': [peak_pow]
        })

        
        all_data.append(df)
    
    # Combine all data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df
# *******************  END Function  *************************

folder_path = r'C:\work_folder\RFB2\Catheter_testing\20250904_IQC_Brush'
# read summary csv files
text1= '.csv'
summary_df= read_and_store_data(folder_path, text1)
writepath= folder_path + '\output.csv'
summary_df.to_csv(writepath)

19316.csv
Value in 2nd column of line 14:  3.520000e+01
19316w1p.csv
Value in 2nd column of line 14:  3.490000e+01
19316w2p.csv
Value in 2nd column of line 14:  3.460000e+01
19316w3p.csv
Value in 2nd column of line 14:  3.430000e+01
23890.csv
Value in 2nd column of line 14:  3.440000e+01
238901p.csv
Value in 2nd column of line 14:  3.410000e+01
23890w2p.csv
Value in 2nd column of line 14:  3.370000e+01
23890w3p.csv
Value in 2nd column of line 14:  3.370000e+01
24603.csv
Value in 2nd column of line 14:  3.650000e+01
24603w1p.csv
Value in 2nd column of line 14:  36
24603w2p.csv
Value in 2nd column of line 14:  3.570000e+01
24603w3p.csv
Value in 2nd column of line 14:  3.520000e+01


<>:57: SyntaxWarning: invalid escape sequence '\o'
<>:57: SyntaxWarning: invalid escape sequence '\o'
C:\Users\Santosh.Behera\AppData\Local\Temp\ipykernel_12440\1492457581.py:57: SyntaxWarning: invalid escape sequence '\o'
  writepath= folder_path + '\output.csv'


2025-09-05 15:13:55


23690.csv
Value in 2nd column of line 14:  3.530000e+01
23690w1p.csv
Value in 2nd column of line 14:  35
23690w2p.csv
Value in 2nd column of line 14:  3.460000e+01
23690w3p.csv
Value in 2nd column of line 14:  3.420000e+01
24603.csv
Value in 2nd column of line 14:  3.760000e+01
24603w1p.csv
Value in 2nd column of line 14:  3.710000e+01
24603w2p.csv
Value in 2nd column of line 14:  3.670000e+01
24603w3p.csv
Value in 2nd column of line 14:  3.620000e+01


<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
C:\Users\Santosh.Behera\AppData\Local\Temp\ipykernel_33528\4031658145.py:1: SyntaxWarning: invalid escape sequence '\o'
  writepath= folder_path + '\output.csv'


In [ ]:
summary_df["sample_repeat_no"].unique()

In [ ]:
summary_df= summary_df[summary_df["Eff"]>60]
create_gui2(summary_df)

In [ ]:
summary_df["SN"].unique()

In [ ]:
summary_df.head

In [ ]:
summary_df.to_csv("writepath")

In [ ]:
df1= pd.read_csv('writepath')
df1.head

In [ ]:
writepath

In [ ]:
writepath= r'C:\work_folder\Misc_codes\prelim_gage\Data\output.csv'
summary_df.to_csv(writepath)